# Tratamiento de imágenes - 2023 - Entregable 2

# Fecha de entrega: 5/5/2023

### Importante:   
En todos los ejercicios se espera que se entregue comentarios sobre decisiones tomadas en la implementación así como un análisis de los resultados. Estos comentarios y análisis se pueden entregar en celdas de texto agregadas a los notebooks o en un informe aparte.

Para la entrega se sube un **zip** (no rar, ni otro formato) con:
* El notebook del entregable modificado. El notebook debe tener las celdas ejecutadas con las salidas correspondientes. Debe tener todo lo necesario para poder ser ejecutado nuevamente
* Todo el resto del contenido del zip del entregable. Si, para mostrar resultados, usan en el notebook imágenes propias, deben incluirlas en la carpeta **imagenes**  del zip. 
* El informe en pdf en caso de no incluir las discusiones dentro del notebook en celdas "Markdown", o si les resulta más sencillo realizar el análisis en un documento aparte.

No se deben incluir en el zip de la entrega las imágenes de prueba que están disponibles en la página web del curso. Se asumirá que esas imágenes están en un directorio y se hará referencia a dichas imágenes en forma relativa a ese directorio (ver la celda siguiente).

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread, imsave

# FORMAS DE VER LAS GRAFICAS --------------------
# ELEGIR UNA DE LAS OPCIONES Y DES-COMENTAR (sacar # de la linea)
# ----------------
# a) graficas en línea  entre las celdas (no interactivo)
%matplotlib inline
# ---------------- 
# b) graficas en línea  entre las celdas (interactivo)
# %matplotlib notebook
# ----------------
# c) graficas en ventanas externas (abre una ventana por cada figura)
# %matplotlib
# ----------------
# d) Si se usa "jupyter lab" en lugar de "jupyter notebook" usar %matplotlib widget en lugar de %matplotlib notebook 
# requiere instalar el modulo "ipympl". Ver https://stackoverflow.com/questions/51922480/javascript-error-ipython-is-not-defined-in-jupyterlab#56416229
# %matplotlib widget
#---------------------------------------------------



# Directorio con la base de imágenes de prueba (las que están en la web de Timag)
# Ej. si usan la imagen trapo.jpg en el notebook no tienen que subirla al eva
# La llaman en el notebook en forma relativa a donde tengan esas imagenes:
# 
# filename = os.path.join(dir_base_imagenes, 'trapo.jpg')
# I = imread(filename)
dir_base_imagenes = '../base_imagenes'


# Directorio con la base de imágenes de documentos (para el ejercicio 5)
# En el notebook acceden a esos datos en forma relativa a donde tengan esas imagenes:
# 
# filename = os.path.join(dir_base_documentos, '0012183661.tif')
# I = imread(filename)
dir_base_documentos = '../base_documentos'


## Transformaciones geométricas

### 1. Implementar una función que realice transformaciones proyectivas

In [ ]:
def bilineal(I, punto, color_de_fondo):
    """
    Dado un punto de coordenadas no necesariamente enteras y una imagen I, 
    retorna la interpolación bilineal entre los cuatro pixeles de I más cercanos
    al punto de entrada.
    
    Args:
    - I (array 2D): Imagen de referencia para interpolar.
    - punto (tuple/list): coordenadas en imagen en formato: punto = (fila,columna) ó punto = [fila,columna]
    - color_de_fondo (int): en caso de que el punto está fuera de las dimensiones de la imagen,
    se le asigna este valor por defecto.
    
    Returns:
    valor_interpolado (int): valor interpolado en escala de grises para el punto. 
    """
    #implementar
    
    return valor_interpolado

def vecino(img, punto, color_de_fondo):
    """
    Dado un punto de coordenadas no necesariamente enteras y una imagen I, 
    retorna el nivel de gris correspondiente al pixel más cercano I.
    
    Args:
    - I (array 2D): Imagen de referencia para interpolar.
    - punto (tuple/list): coordenadas en imagen en formato: punto = (fila,columna) ó punto = [fila,columna]
    - color_de_fondo (int): en caso de que el punto está fuera de las dimensiones de la imagen,
    se le asigna este valor por defecto.
    
    Returns:
    valor_interpolado (int): valor interpolado en escala de grises para el punto. 
    """

    #implementar
    
    return valor_interpolado

def transformar(I, T, tipo_interpolacion='vecino', color_de_fondo=0):
    """
    Aplicar una transformacion geometrica T a imagen I, especificando el tipo de interpolacion.
    
    Args:
    - I (array 2D): imagen a transformar.
    - T (array 2D): matriz de transformacion de tamaño 3x3.
    - tipo_interpolacion (str): interpolación a utilizar: 'vecino' más cercano (default) o 'bilineal'.
    - color_de_fondo (int): en caso de que el punto está fuera de las dimensiones de la imagen,
    se le asigna este valor por defecto.
    
    Returns:
    I_transformada (array 2D): imagen transformada
    
    """
    #implementar
    
    return I_transformada

### 2. Usando la función transformar, mostrar ejemplos de transformaciones isométricas, afines y proyectivas  con diferentes interpolaciones

Parecido a lo que se muestra en estos ejemplos de scikit-image:  
https://scikit-image.org/docs/stable/auto_examples/transform/plot_transform_types.html


#### Mostrar una rotación con origen en vértice superior izquierdo de la imagen (origen de coordenadas) 

#### Mostrar una rotación con origen en el centro de la imagen
La transformación de rotación en el centro de la imagen puede calcularse como la composición de una traslación, una rotación y una traslación.  

In [ ]:
# Muestre las matrices correspondientes a las tres transformaciones 



# Calcule la matriz de rotación en el centro como la composición de las tres transformaciones anteriores



# Aplique la transformación y muestre el resultado



#### Mostrar una transformación afin 

#### Mostrar una transformación proyectiva

### 3. Implementar el cálculo de una matriz de transformación proyectiva a partir de un conjunto de  4 o más puntos correspondientes
Para chequear su implementación se puede comparar el resultado con cv2.findHomography por ejemplo


In [ ]:
# Entradas:
#    puntosA   Nx2
#    puntosB   Nx2
#    con N>=4
# Salida
#    H         3x3    
def calcular_homografia(puntosA, puntosB):
    #implementar
    
    return H
    

#### Chequear la función implementada

In [ ]:
# chequear la función calcularHomografía con un conjunto de puntos N=4
origen =  np.array([(0,0), (0,100), (100, 100), (100, 0)])
destino = np.array([(0,0), (0,100), (100, 50),  (100, 0)])
print(origen)
print(destino)

# calcular la homografía


# aplicar a los puntos origen y verificar si el resultado son los puntos destino




In [ ]:
# chequear la función calcularHomografía con un conjunto de puntos N>4
origen =  np.array([(0,0), (0,52), (0,100), (100, 100), (100, 0)])
destino = np.array([(0,0), (0,48), (0,100), (100, 50),  (100, 0)])
print(origen)
print(destino)


# calcular la homografía


# aplicar a los puntos origen y verificar si el resultado es cercano a los puntos destino


## Segmentación

### 4. Detección de líneas con la transformada de Hough

   
Se puede encontrar una buena explicación y pseudocódigo en los libros de [Burger & Burge](https://books.google.com.uy/books?id=YpzWCwAAQBAJ&printsec=frontcover#v=onepage&q&f=false).   
Se sugiere seguir dicho pseudocódigo para la implementación.

#### Dibujo de líneas

In [ ]:
def plot_lineas_hough(fig, L)
    '''
    Grafica sobre la figura especificada las líneas definidas en L

    Args
    ----
    fig : Numero de la figura de matplotlib donde se plotean las líneas
    L :   [[theta, rho, votos], [theta, rho, votos], .....]
          Lista de rectas detectadas con la función lineas_hough 

    Returns
    -------
    Nothing

    '''
    
    # Implementar

#### Mostrar el funcionamiento de plot_lineas_hough para distintos valores theta-rho
Notar que se puede implementar y probar **plot_lineas_hough** antes de implementar **lineas_hough**.  
Se sugiere dejar andando correctamente **plot_lineas_hough** primero. Esto permitirá luego probar rápidamente los resultados de **lineas_hough**.
  

In [ ]:
# Imagen vacía a plottear
zeros = np.zeros((250,250))

# líneas horizontal y vertical que pasan por la mitad de la imagen
lineas = [[0,0],[0,np.pi/2]]    
# lineas separadas por un ángulo de pi/8 radianes
lineas2 = [[0,0],[0,np.pi/8],[0,np.pi/4],[0,3*np.pi/8],[0,np.pi/2],[0,5*np.pi/8],[0,3*np.pi/4],[0,7*np.pi/8]] 

# Inicializar figura
fig, ax = plt.subplots(1,2,figsize=(8,3))
# Plot lineas
plot_lineas_hough(ax[0],zeros, lineas)
ax[0].set_title("Lineas")
# Plot lineas2
plot_lineas_hough(ax[1],zeros, lineas2)
ax[1].set_title("Otras lineas")
fig.tight_layout()
plt.show()

Las imágenes de salida deberían ser del estilo:


![Alt text](imagenes/output_plot_hough.png)

#### Detección de líneas

In [ ]:

    

def lineas_hough(I, m, n, minimo_votos, debug=False):
    '''
    Detección de líneas mediante la Transformada de Hough
    
    Args
    ----
    I : imagen binaria (resultado de un detector de bordes por ejemplo)
    m, n : Enteros que determinan las dimensiones del acumulador
           m: cantidad de pasos en Theta
           n: cantidad de pasos en Rho
    minimo_votos : Cantidad mínima de votos que debe tener una recta 
    debug : Booleano opcional. En caso de ser verdadero, se muestra la imagen del acumulador
    
    Returns
    -------
    L : [[theta, rho, votos], [theta, rho, votos], .....]
        Lista con las rectas detectadas donde cada elemento de la lista es una terna [theta, rho, votos]
        La lista está ordenada en forma decreciente en la cantidad de votos 

    '''
    
    # Implementar
    
    return L



#### Mostrar el funcionamiento de lineas_hough sobre imágenes artificiales 
Para cada imagen artificial:   
a) detectar líneas  
b) mostrar el acumulador y plotear las rectas    
c) relacionar el contenido del acumulador con las rectas correspondientes    

In [ ]:
#Ejemplo 1 
I = np.zeros((200,300), dtype=np.uint8)
I[50,50]=255
I[50,250]=255

plt.figure()
plt.imshow(I)


In [ ]:
#Ejemplo 2
I = np.zeros((200,300), dtype=np.uint8)
I[50,50]=255
I[150,50]=255

plt.figure()
plt.imshow(I)

In [ ]:
#Ejemplo 3
I = np.zeros((200,300), dtype=np.uint8)
I[50,50]=255
I[150,150]=255

plt.figure()
plt.imshow(I)

In [ ]:
# Ejemplo 4
I = np.zeros((200,300), dtype=np.uint8)
I[50,50]=255
I[50,250]=255
I[150,150]=255

plt.figure()
plt.imshow(I)


#### Mostrar el funcionamiento de lineas_hough sobre la imagen trapecio.png
Comentar resultados al variar la resolución en el espacio de parámetros (m y n en la función lineas_hough)

### 5. Aplicación: Segmentación de imágenes de documentos

En esta parte se usará un subset de la base RVL-CDIP que contiene imágenes de documentos escaneados.
https://adamharley.com/rvl-cdip/  


Utilizando herramientas vistas en el curso (ej. análisis en frecuencia, transformaciones geométricas, segmentación por umbral, detección de bordes, lineas y regiones, morfología, etiquetado de regiones, entre otros) se buscará identificar en cada imagen:  

**Tareas**

1. Orientación general del documento en la imagen.  ¿El documento está en formato vertical o está apaisado? 
2. Ángulo de giro. Ángulo a girar el documento en sentido horario para que las líneas y/o texto queden horizontales.  
3. ¿El documento es un formulario o tabla? Se entenderá por formulario o tabla un documento que cuente con algunas líneas verticales y horizontales. 
4. ¿El documento tiene columnas de texto? El documento tiene agrupaciones de texto en columnas por varias filas.  
5. Límites horizontales del documento. Indicar la columna de inicio y fin del núcleo principal del documento. En caso que (4) sea afirmativo, indicar también las posiciones horizontales que separan las columnas de texto.    
6. Alto de líneas. Distancia vertical en píxeles entre la línea de base de una fila y la de la siguiente.  
7. Ancho de letra. En los casos que el documento tenga un tipo de letra "monospaced" (https://en.wikipedia.org/wiki/Monospaced_font) se podrá determinar el ancho de letra como la distancia horizontal en píxeles entre el comienzo de una letra y de la siguiente. 
8. ¿El documento es sólo texto y/o líneas o contiene alguna figura? Se entenderá por figura un conjunto conexo de píxeles mayor al 1% del área de la imagen y que no está compuesto por texto/líneas y que tampoco es una región quasi-uniforme (ej. un rectángulo quasi-uniforme no se considera una figura)

En el pdf adjunto a este notebook hay ejemplos de las imágenes de documentos y de los resultados esperados para cada una de las tareas. 


**Herramientas**  
Para la resolución de esta parte se podrá usar funciones de numpy, scipy, scikit-image, de OpenCV y herramientas de Fiji. 

**Resultados esperados**
* El problema se plantea en forma abierta y por lo tanto no hay una forma única de resolver cada tarea. La idea es que exploren las distintas técnicas y algoritmos vistos a lo largo del curso sobre un problema concreto
* El conjunto de imágenes que les dejamos disponibles tiene cierta variabilidad por lo que, para las distintas tareas, habrá algunas imágenes más complicadas que otras. No esperen un funcionamiento perfecto de sus soluciones. 
* Para cada tarea implementada, 
  * se valorará que comenten él o los posibles métodos que la podrían resolver, antes de realizar una implementación concreta
  * implementen alguno de esos métodos en la función cuyo esqueleto le pasamos. Pueden hacer funciones auxiliares si lo necesitan. Alternativamente pueden hacer un procedimiento o una macro si usan Fiji.
  * muestren tests de su método sobre algunas imágenes
  * opcionalmente pueden mostrar, si la tarea lo permite, resultados cuantitativos sobre el conjunto de imágenes (por ejemplo el porcentaje de acierto)
  * comenten en qué casos funciona y qué casos son difíciles para el método (no es necesario un listado exhaustivo)  
* Implementaciones  
  * No se espera que un método implementado para una tarea funcione para todas o casi todas las imágenes. 
  * Las tareas tienen diferente dificultad. Para algunas será más sencillo hacer un método general y para otras se podrá hacer algo que funcione para un conjunto más restringido de imágenes. Hagan métodos no muy extensos, los testean y discuten resultados.
  * Realizar al menos 5 de las 8 tareas
   
  



##### 5.1 Orientación general

Discusión de métodos, pruebas

Implementación

In [11]:
def tiene_orientacion_vertical(I_doc):
    """
    Determina la orientación del documento en la imagen.

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    orientacion_vertical (bool): Orientación del documento, puede ser 'vertical' (True) o 'apaisado' (False)
    """

    # IMPLEMENTAR #

    return orientacion_vertical

Tests sobre imágenes

##### 5.2 Ángulo de giro

Discusión de métodos, pruebas

Implementación

In [ ]:
def angulo_de_giro(I_doc):
    """
    Determina el ángulo (en grados) a girar la imagen en sentido horario 
    para que las líneas y/o texto del documento queden horizontales.

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    angulo (double): ángulo en grados
    """

    # IMPLEMENTAR #

    return angulo

Tests sobre imágenes

##### 5.3 Es un formulario o tabla ?

Discusión de métodos, pruebas

Implementación

In [ ]:
def es_tipo_tabla(I_doc):
    """
    Determina si el documento en la imagen es tipo formulario/tabla

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    es_tabla (bool): Verdadero si el documento en la imagen es tipo formulario/tabla
    """

    # IMPLEMENTAR #

    return es_tabla

Tests sobre imágenes

##### 5.4 ¿El documento tiene texto en columnas?

Discusión de métodos, pruebas

Implementación

In [ ]:
def tiene_texto_en_columnas(I_doc):
    """
    Determina si el documento tiene texto en columnas

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    tiene_columnas (bool): Verdadero en caso de tener columnas
    """

    # IMPLEMENTAR #

    return tiene_columnas

Tests sobre imágenes

##### 5.5 Límites horizontales

Discusión de métodos, pruebas

Implementación

In [ ]:
def limites_horizontales(I_doc):
    """
    Devuelve una lista de valores con las columnas de inicio y fin del contenido del documento
    y, si contiene columnas, las posiciones horizontales que separan las columnas

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    limites (list): [inicio, fin] si el documento no contiene texto en columnas
                    [inicio, sep1,...,sepN, fin] si contiene columnas 
    """

    # IMPLEMENTAR #

    return limites

Tests sobre imágenes

##### 5.6 Alto de línea

Discusión de métodos, pruebas

Implementación

In [ ]:
def alto_de_linea(I_doc):
    """
    Devuelve el alto de línea del documento

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    alto (int o double): Alto de la línea 
    """

    # IMPLEMENTAR #

    return alto

Tests sobre imágenes

##### 5.7 Ancho de letra

Discusión de métodos, pruebas

Implementación

In [10]:
def ancho_de_letra(I_doc):
    """
    Devuelve el ancho de letra del documento
    Se supone que la imagen pasada comprende un documento que, mayoritariamente,
    tiene tipo de letra "monospaced" 

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    ancho (int o double): ancho de letra 
    """

    # IMPLEMENTAR #

    return ancho

Tests sobre imágenes

##### 5.8 ¿El documento contiene figuras?

Discusión de métodos, pruebas

Implementación

In [ ]:
def tiene_alguna_figura(I_doc):
    """
    Determina si el documento tiene alguna figura

    Args:
    -----
      I_doc (2D array): imagen con un documento escaneado.

    Returns:
    --------
    tiene_figura (bool): Verdadero en caso de tener alguna figura
    """

    # IMPLEMENTAR #

    return tiene_figura

Tests sobre imágenes